# TCGA Gene Expression Analysis

This notebook demonstrates how to download and analyze real cancer gene expression data from TCGA (The Cancer Genome Atlas).

We'll use the **TCGAbiolinks** approach via Python, or alternatively use **GDC Data Portal** API.

## Methods covered:
1. Download TCGA data using GDC API
2. Process RNA-seq count data
3. Normalize expression data
4. Differential expression analysis
5. Visualization and interpretation

In [ ]:
# Install required packages
# Run this cell first if packages are not installed
#!pip install requests pandas numpy matplotlib seaborn scipy scikit-learn statsmodels adjustText

In [1]:
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.multitest import multipletests
import os
import gzip
import tarfile
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Download TCGA Data from GDC Portal

We'll download gene expression data for a specific cancer type.
Example: TCGA-BRCA (Breast Cancer)

In [2]:
class TCGADataDownloader:
    """
    Download TCGA data from GDC Data Portal
    """
    
    def __init__(self):
        self.files_endpt = "https://api.gdc.cancer.gov/files"
        self.data_endpt = "https://api.gdc.cancer.gov/data"
        self.cases_endpt = "https://api.gdc.cancer.gov/cases"
        
    def query_files(self, project_id, data_type="Gene Expression Quantification", 
                    workflow_type="STAR - Counts", sample_type=None, size=100):
        """
        Query files from GDC
        
        Parameters:
        - project_id: e.g., 'TCGA-BRCA'
        - data_type: type of data
        - workflow_type: analysis workflow
        - sample_type: 'Primary Tumor', 'Solid Tissue Normal', etc.
        - size: number of files to retrieve
        """
        
        filters = {
            "op": "and",
            "content": [
                {
                    "op": "in",
                    "content": {
                        "field": "cases.project.project_id",
                        "value": [project_id]
                    }
                },
                {
                    "op": "in",
                    "content": {
                        "field": "files.data_type",
                        "value": [data_type]
                    }
                },
                {
                    "op": "in",
                    "content": {
                        "field": "files.analysis.workflow_type",
                        "value": [workflow_type]
                    }
                }
            ]
        }
        
        # Add sample type filter if specified
        if sample_type:
            filters["content"].append({
                "op": "in",
                "content": {
                    "field": "cases.samples.sample_type",
                    "value": [sample_type]
                }
            })
        
        params = {
            "filters": json.dumps(filters),
            "fields": "file_id,file_name,cases.case_id,cases.samples.sample_type,cases.project.project_id",
            "format": "JSON",
            "size": str(size)
        }
        
        response = requests.get(self.files_endpt, params=params)
        
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error: {response.status_code}")
            return None
    
    def download_file(self, file_id, output_dir="tcga_data"):
        """
        Download a single file from GDC
        """
        os.makedirs(output_dir, exist_ok=True)
        
        params = {"ids": file_id}
        response = requests.get(self.data_endpt, params=params, stream=True)
        
        if response.status_code == 200:
            output_path = os.path.join(output_dir, f"{file_id}.tar.gz")
            
            with open(output_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            
            # Extract the tar.gz file
            with tarfile.open(output_path, 'r:gz') as tar:
                tar.extractall(output_dir)
            
            # Remove the tar.gz file
            os.remove(output_path)
            
            return True
        else:
            print(f"Failed to download {file_id}: {response.status_code}")
            return False

# Initialize downloader
downloader = TCGADataDownloader()

print("TCGA Data Downloader initialized successfully!")

TCGA Data Downloader initialized successfully!


## 2. Query and Download TCGA Data

Let's download breast cancer (BRCA) data - both tumor and normal samples

In [3]:
# Query tumor samples
print("Querying Primary Tumor samples...")
tumor_files = downloader.query_files(
    project_id="TCGA-BRCA",
    sample_type="Primary Tumor",
    size=20  # Download 20 tumor samples
)

# Query normal samples
print("Querying Normal Tissue samples...")
normal_files = downloader.query_files(
    project_id="TCGA-BRCA",
    sample_type="Solid Tissue Normal",
    size=20  # Download 20 normal samples
)

if tumor_files and normal_files:
    print(f"\nFound {len(tumor_files['data']['hits'])} tumor samples")
    print(f"Found {len(normal_files['data']['hits'])} normal samples")
else:
    print("Error querying files")

Querying Primary Tumor samples...
Querying Normal Tissue samples...

Found 20 tumor samples
Found 20 normal samples


In [4]:
# Download the files (this may take several minutes)
print("Downloading tumor samples...")
tumor_metadata = []

for hit in tumor_files['data']['hits'][:20]:  # Limit to 20 for demo
    file_id = hit['file_id']
    case_id = hit['cases'][0]['case_id']
    sample_type = hit['cases'][0]['samples'][0]['sample_type']
    
    print(f"Downloading {file_id}...")
    success = downloader.download_file(file_id)
    
    if success:
        tumor_metadata.append({
            'file_id': file_id,
            'case_id': case_id,
            'sample_type': sample_type,
            'condition': 'Tumor'
        })

print("\nDownloading normal samples...")
normal_metadata = []

for hit in normal_files['data']['hits'][:20]:  # Limit to 20 for demo
    file_id = hit['file_id']
    case_id = hit['cases'][0]['case_id']
    sample_type = hit['cases'][0]['samples'][0]['sample_type']
    
    print(f"Downloading {file_id}...")
    success = downloader.download_file(file_id)
    
    if success:
        normal_metadata.append({
            'file_id': file_id,
            'case_id': case_id,
            'sample_type': sample_type,
            'condition': 'Normal'
        })

# Combine metadata
all_metadata = tumor_metadata + normal_metadata
metadata_df = pd.DataFrame(all_metadata)

print(f"\nDownload complete! Total samples: {len(metadata_df)}")
print(metadata_df.head())

Failed to download 744a6d3d-b666-49aa-8d26-47f34e3d1eb5: 405
Failed to download 4ecc1f1a-8ff4-4552-a5e8-7a9652b6d1d5: 405
Failed to download 1ace2a0c-773d-45b5-8fd6-968c88731bbb: 405
Failed to download 2d5b0962-b5c4-4482-9f28-47e4dcdb6df6: 405
Failed to download c2515ef2-5cfc-4cc2-9541-ae072616d361: 405
Failed to download 61647238-a8c4-4156-af15-c32947983cfd: 405
Failed to download 488399aa-ef3c-42d7-817f-9db11b9902dd: 405
Failed to download 3d0b3b80-92ae-44dc-89fc-2ead925b81a2: 405
Failed to download 5cce30f3-99ae-4959-987a-0d280dc2ec44: 405
Failed to download ddeb1445-08ae-441c-9f2a-4647df1b4ed6: 405
Failed to download 5c0579f7-6184-4afa-805c-7aeec4a4c5d2: 405
Failed to download 9c00a43c-1da0-4e3b-b7e2-54a48d29ecae: 405
Failed to download 6ce774a5-2636-4a01-ab6e-737f89a4bd4f: 405
Failed to download fed0a382-2eee-4acc-af00-53a9fdcd2a61: 405
Failed to download 80d565a0-59f6-409e-8850-5d4e2b5b996d: 405
Failed to download 263123b0-ec68-47cd-96dd-4f0795503d89: 405
Failed to download f205f

## 3. Load and Process Gene Expression Data

In [ ]:
def load_tcga_counts(data_dir="tcga_data"):
    """
    Load TCGA gene expression count data from downloaded files
    """
    expression_data = {}
    
    # Find all count files
    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file.endswith('.tsv') or file.endswith('.counts'):
                file_path = os.path.join(root, file)
                
                # Read the count file
                df = pd.read_csv(file_path, sep='\t', comment='#')
                
                # TCGA STAR counts files have specific format
                # Columns: gene_id, gene_name, gene_type, unstranded, stranded_first, stranded_second
                if 'gene_id' in df.columns:
                    # Use unstranded counts or stranded_first
                    count_column = 'unstranded' if 'unstranded' in df.columns else 'stranded_first'
                    
                    # Get file_id from directory name
                    file_id = os.path.basename(root)
                    
                    # Store counts with gene names as index
                    if 'gene_name' in df.columns:
                        counts = df.set_index('gene_name')[count_column]
                    else:
                        counts = df.set_index('gene_id')[count_column]
                    
                    expression_data[file_id] = counts
    
    # Combine all samples into a single DataFrame
    expression_df = pd.DataFrame(expression_data)
    
    return expression_df

# Load the data
print("Loading gene expression data...")
raw_counts = load_tcga_counts()

print(f"\nExpression matrix shape: {raw_counts.shape}")
print(f"Number of genes: {raw_counts.shape[0]}")
print(f"Number of samples: {raw_counts.shape[1]}")
print("\nFirst few rows and columns:")
print(raw_counts.iloc[:5, :5])

## 4. Filter and Normalize Gene Expression Data

In [ ]:
def filter_low_expression_genes(counts_df, min_count=10, min_samples=3):
    """
    Filter out genes with very low expression
    Keep genes with at least min_count reads in at least min_samples
    """
    mask = (counts_df >= min_count).sum(axis=1) >= min_samples
    filtered_df = counts_df[mask]
    
    print(f"Filtered from {counts_df.shape[0]} to {filtered_df.shape[0]} genes")
    return filtered_df

def normalize_counts(counts_df, method='log2_cpm'):
    """
    Normalize count data
    
    Methods:
    - log2_cpm: log2(counts per million + 1)
    - tpm: Transcripts per million (requires gene lengths)
    """
    if method == 'log2_cpm':
        # Calculate CPM (counts per million)
        lib_sizes = counts_df.sum(axis=0)
        cpm = counts_df.div(lib_sizes, axis=1) * 1e6
        
        # Log2 transform with pseudocount
        log2_cpm = np.log2(cpm + 1)
        
        return log2_cpm
    
    else:
        raise ValueError(f"Unknown normalization method: {method}")

# Filter low expression genes
print("Filtering low expression genes...")
filtered_counts = filter_low_expression_genes(raw_counts)

# Normalize
print("\nNormalizing expression data...")
normalized_expression = normalize_counts(filtered_counts)

print(f"\nNormalized expression matrix shape: {normalized_expression.shape}")
print("\nExpression distribution:")
print(normalized_expression.describe())

## 5. Match Samples with Metadata

In [ ]:
# Match sample IDs between expression data and metadata
common_samples = list(set(normalized_expression.columns) & set(metadata_df['file_id']))

print(f"Samples in both expression data and metadata: {len(common_samples)}")

# Subset expression data
expression_subset = normalized_expression[common_samples]

# Subset and reorder metadata
metadata_subset = metadata_df[metadata_df['file_id'].isin(common_samples)].copy()
metadata_subset = metadata_subset.set_index('file_id').loc[common_samples].reset_index()

print(f"\nFinal dataset: {expression_subset.shape[0]} genes × {expression_subset.shape[1]} samples")
print(f"\nSample distribution:")
print(metadata_subset['condition'].value_counts())

## 6. Quality Control Visualization

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Sample library sizes
lib_sizes = raw_counts[common_samples].sum(axis=0) / 1e6  # in millions
colors = ['red' if metadata_subset[metadata_subset['file_id'] == s]['condition'].values[0] == 'Tumor' 
          else 'blue' for s in common_samples]

axes[0].bar(range(len(lib_sizes)), lib_sizes, color=colors, alpha=0.7)
axes[0].set_xlabel('Sample')
axes[0].set_ylabel('Library Size (millions)')
axes[0].set_title('Library Sizes by Sample')
axes[0].axhline(lib_sizes.median(), color='black', linestyle='--', label='Median')
axes[0].legend()

# Expression distribution
tumor_samples = metadata_subset[metadata_subset['condition'] == 'Tumor']['file_id'].values
normal_samples = metadata_subset[metadata_subset['condition'] == 'Normal']['file_id'].values

if len(tumor_samples) > 0 and len(normal_samples) > 0:
    axes[1].boxplot(
        [expression_subset[tumor_samples].values.flatten(),
         expression_subset[normal_samples].values.flatten()],
        labels=['Tumor', 'Normal']
    )
    axes[1].set_ylabel('Expression (log2 CPM)')
    axes[1].set_title('Expression Distribution by Condition')

plt.tight_layout()
plt.show()

## 7. Principal Component Analysis (PCA)

In [ ]:
# Standardize and perform PCA
scaler = StandardScaler()
scaled_data = scaler.fit_transform(expression_subset.T)

pca = PCA(n_components=10)
pca_result = pca.fit_transform(scaled_data)

# Create PCA DataFrame
pca_df = pd.DataFrame(
    pca_result,
    columns=[f'PC{i+1}' for i in range(10)],
    index=expression_subset.columns
)

# Add condition information
pca_df = pca_df.merge(
    metadata_subset[['file_id', 'condition']].set_index('file_id'),
    left_index=True,
    right_index=True
)

# Plot PCA
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# PC1 vs PC2
for condition, color in [('Tumor', 'red'), ('Normal', 'blue')]:
    mask = pca_df['condition'] == condition
    axes[0].scatter(
        pca_df.loc[mask, 'PC1'],
        pca_df.loc[mask, 'PC2'],
        c=color,
        label=condition,
        s=100,
        alpha=0.7,
        edgecolors='black'
    )

axes[0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)')
axes[0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)')
axes[0].set_title('PCA: TCGA BRCA Samples')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Scree plot
axes[1].bar(range(1, 11), pca.explained_variance_ratio_ * 100)
axes[1].set_xlabel('Principal Component')
axes[1].set_ylabel('Variance Explained (%)')
axes[1].set_title('Scree Plot')
axes[1].set_xticks(range(1, 11))

plt.tight_layout()
plt.show()

print(f"Cumulative variance explained by first 3 PCs: {pca.explained_variance_ratio_[:3].sum()*100:.1f}%")

## 8. Differential Expression Analysis

In [ ]:
def perform_differential_expression_tcga(expression_df, metadata_df):
    """
    Perform differential expression analysis on TCGA data
    """
    results = []
    
    # Get sample IDs for each condition
    tumor_samples = metadata_df[metadata_df['condition'] == 'Tumor']['file_id'].values
    normal_samples = metadata_df[metadata_df['condition'] == 'Normal']['file_id'].values
    
    # Filter to samples present in expression data
    tumor_samples = [s for s in tumor_samples if s in expression_df.columns]
    normal_samples = [s for s in normal_samples if s in expression_df.columns]
    
    print(f"Analyzing {len(tumor_samples)} tumor vs {len(normal_samples)} normal samples")
    
    for gene in expression_df.index:
        tumor_expr = expression_df.loc[gene, tumor_samples].values
        normal_expr = expression_df.loc[gene, normal_samples].values
        
        # Calculate statistics
        mean_tumor = np.mean(tumor_expr)
        mean_normal = np.mean(normal_expr)
        log2fc = mean_tumor - mean_normal  # Already in log2 space
        
        # T-test
        t_stat, p_value = stats.ttest_ind(tumor_expr, normal_expr)
        
        results.append({
            'Gene': gene,
            'Mean_Tumor': mean_tumor,
            'Mean_Normal': mean_normal,
            'Log2FC': log2fc,
            'T_statistic': t_stat,
            'P_value': p_value
        })
    
    results_df = pd.DataFrame(results)
    
    # Multiple testing correction
    _, results_df['FDR'], _, _ = multipletests(
        results_df['P_value'],
        method='fdr_bh'
    )
    
    # Add significance label
    results_df['Significant'] = (results_df['FDR'] < 0.05) & (abs(results_df['Log2FC']) > 1)
    
    # Sort by p-value
    results_df = results_df.sort_values('P_value')
    
    return results_df

# Perform differential expression
print("Performing differential expression analysis...")
de_results = perform_differential_expression_tcga(expression_subset, metadata_subset)

# Summary
n_significant = de_results['Significant'].sum()
n_upregulated = ((de_results['Significant']) & (de_results['Log2FC'] > 0)).sum()
n_downregulated = ((de_results['Significant']) & (de_results['Log2FC'] < 0)).sum()

print(f"\n{'='*60}")
print(f"DIFFERENTIAL EXPRESSION RESULTS (TCGA BRCA)")
print(f"{'='*60}")
print(f"Total genes analyzed: {len(de_results)}")
print(f"Significant genes (FDR < 0.05, |Log2FC| > 1): {n_significant}")
print(f"Upregulated in tumor: {n_upregulated}")
print(f"Downregulated in tumor: {n_downregulated}")
print(f"{'='*60}")

print(f"\nTop 20 differentially expressed genes:")
print(de_results[['Gene', 'Log2FC', 'P_value', 'FDR', 'Significant']].head(20))

## 9. Volcano Plot

In [ ]:
# Prepare data for volcano plot
de_results['-log10(FDR)'] = -np.log10(de_results['FDR'])

# Create volcano plot
plt.figure(figsize=(12, 8))

# Not significant
not_sig = de_results[~de_results['Significant']]
plt.scatter(not_sig['Log2FC'], not_sig['-log10(FDR)'], 
           c='gray', alpha=0.4, s=10, label='Not significant')

# Significant and upregulated
sig_up = de_results[(de_results['Significant']) & (de_results['Log2FC'] > 0)]
plt.scatter(sig_up['Log2FC'], sig_up['-log10(FDR)'], 
           c='red', alpha=0.7, s=30, label=f'Upregulated ({len(sig_up)})')

# Significant and downregulated
sig_down = de_results[(de_results['Significant']) & (de_results['Log2FC'] < 0)]
plt.scatter(sig_down['Log2FC'], sig_down['-log10(FDR)'], 
           c='blue', alpha=0.7, s=30, label=f'Downregulated ({len(sig_down)})')

# Add threshold lines
plt.axhline(-np.log10(0.05), color='black', linestyle='--', linewidth=1, alpha=0.5, label='FDR = 0.05')
plt.axvline(-1, color='black', linestyle='--', linewidth=1, alpha=0.5)
plt.axvline(1, color='black', linestyle='--', linewidth=1, alpha=0.5)

# Annotate top genes
top_genes = de_results.nsmallest(10, 'FDR')
for _, row in top_genes.iterrows():
    plt.annotate(row['Gene'], 
                xy=(row['Log2FC'], row['-log10(FDR)']),
                xytext=(5, 5), textcoords='offset points',
                fontsize=8, alpha=0.7)

plt.xlabel('Log2 Fold Change (Tumor vs Normal)', fontsize=12)
plt.ylabel('-log10(FDR)', fontsize=12)
plt.title('Volcano Plot: TCGA BRCA Differential Expression', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 10. Heatmap of Top Genes

In [ ]:
# Select top 50 differentially expressed genes
top_genes = de_results.nsmallest(50, 'FDR')['Gene'].values

# Subset expression data
heatmap_data = expression_subset.loc[top_genes]

# Z-score normalize
heatmap_data_zscore = heatmap_data.sub(heatmap_data.mean(axis=1), axis=0).div(
    heatmap_data.std(axis=1), axis=0
)

# Create color map for conditions
col_colors = metadata_subset.set_index('file_id')['condition'].map(
    {'Tumor': 'red', 'Normal': 'blue'}
)

# Reorder columns to match
col_colors = col_colors.loc[heatmap_data_zscore.columns]

# Create clustered heatmap
plt.figure(figsize=(14, 10))
g = sns.clustermap(
    heatmap_data_zscore,
    col_colors=col_colors,
    cmap='RdBu_r',
    center=0,
    vmin=-3,
    vmax=3,
    figsize=(14, 10),
    cbar_kws={'label': 'Z-score'},
    yticklabels=True,
    xticklabels=False,
    row_cluster=True,
    col_cluster=True
)
plt.suptitle('Top 50 Differentially Expressed Genes (TCGA BRCA)', 
             y=0.98, fontsize=14, fontweight='bold')
plt.show()

## 11. Save Results

In [ ]:
# Save differential expression results
de_results.to_csv('tcga_brca_differential_expression.csv', index=False)
print("Saved: tcga_brca_differential_expression.csv")

# Save significant genes
significant_genes = de_results[de_results['Significant']]
significant_genes.to_csv('tcga_brca_significant_genes.csv', index=False)
print(f"Saved: tcga_brca_significant_genes.csv ({len(significant_genes)} genes)")

# Save normalized expression matrix
expression_subset.to_csv('tcga_brca_normalized_expression.csv')
print("Saved: tcga_brca_normalized_expression.csv")

# Save metadata
metadata_subset.to_csv('tcga_brca_sample_metadata.csv', index=False)
print("Saved: tcga_brca_sample_metadata.csv")

## 12. Expression Profiles of Cancer-Related Genes

In [ ]:
# Look for known cancer genes in the data
cancer_genes = ['BRCA1', 'BRCA2', 'TP53', 'ESR1', 'ERBB2', 'MYC']
available_cancer_genes = [g for g in cancer_genes if g in expression_subset.index]

if len(available_cancer_genes) > 0:
    print(f"Found {len(available_cancer_genes)} known cancer genes in dataset:")
    print(available_cancer_genes)
    
    # Plot their expression
    n_genes = min(6, len(available_cancer_genes))
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    tumor_samples_list = metadata_subset[metadata_subset['condition'] == 'Tumor']['file_id'].values
    normal_samples_list = metadata_subset[metadata_subset['condition'] == 'Normal']['file_id'].values
    
    for idx, gene in enumerate(available_cancer_genes[:n_genes]):
        gene_expr = expression_subset.loc[gene]
        
        tumor_expr = gene_expr[tumor_samples_list].values
        normal_expr = gene_expr[normal_samples_list].values
        
        # Box plot
        bp = axes[idx].boxplot([tumor_expr, normal_expr], 
                               labels=['Tumor', 'Normal'], 
                               patch_artist=True)
        
        bp['boxes'][0].set_facecolor('red')
        bp['boxes'][1].set_facecolor('blue')
        
        # Add points
        axes[idx].scatter([1]*len(tumor_expr), tumor_expr, alpha=0.3, c='darkred', s=20)
        axes[idx].scatter([2]*len(normal_expr), normal_expr, alpha=0.3, c='darkblue', s=20)
        
        # Get stats
        if gene in de_results['Gene'].values:
            gene_stats = de_results[de_results['Gene'] == gene].iloc[0]
            axes[idx].set_title(
                f"{gene}\nLog2FC={gene_stats['Log2FC']:.2f}, FDR={gene_stats['FDR']:.2e}"
            )
        else:
            axes[idx].set_title(gene)
        
        axes[idx].set_ylabel('Expression (log2 CPM)')
        axes[idx].grid(True, alpha=0.3)
    
    # Hide unused subplots
    for idx in range(n_genes, 6):
        axes[idx].set_visible(False)
    
    plt.tight_layout()
    plt.show()
else:
    print("No known cancer genes found in this dataset")

## Summary

This notebook demonstrated:

1. ✅ **Downloading real TCGA data** using the GDC API
2. ✅ **Processing RNA-seq count data** from TCGA
3. ✅ **Quality control and normalization** (CPM, log2 transformation)
4. ✅ **PCA analysis** to visualize sample relationships
5. ✅ **Differential expression analysis** (tumor vs normal)
6. ✅ **Multiple testing correction** (FDR)
7. ✅ **Publication-quality visualizations** (volcano plots, heatmaps)
8. ✅ **Analysis of cancer-related genes**

### Next Steps:

- Download more samples for increased statistical power
- Analyze different cancer types (TCGA-LUAD, TCGA-COAD, etc.)
- Perform pathway enrichment analysis (GSEA, GO, KEGG)
- Integrate with clinical data (survival analysis)
- Use DESeq2 (via rpy2) for more sophisticated normalization
- Analyze subgroups (e.g., ER+/ER-, HER2+/HER2- in breast cancer)